In [1]:
# import API keys
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a.env file
_ = load_dotenv(find_dotenv())

openai_api_key = os.environ["OPENAI_API_KEY"]

# Load opensource api models
groql_api_key = os.environ["GROQ_API_KEY"]

### RunnablePassThrough
* It does no do anything to the input data
* Let's see it in a very simple example: a chain with a RunnablePassThrough() will output the original output withouut any modifications.

In [14]:
# import chat model
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


llamaChatModel = ChatGroq(
    model="llama3-70b-8192"
)

chain = RunnablePassthrough()

In [5]:
# Input data returned without modifications
chain.invoke("Paul N")

'Paul N'

### RunnableLamda
* To use a custom function inside a LCEL chain we need to wrap it up with a RunnableLamda
* Let's define a very simple function to create Russian lastnames

In [6]:
# Custom function
def russian_lastname(name:str) -> str:
    return f"{name}ovich"

In [7]:
# Test custom function outside of chains
russian_last = russian_lastname("Paul")
russian_last

'Paulovich'

In [8]:
# import RunnableLambda
from langchain_core.runnables import RunnableLambda


In [9]:
# Wrap custom function in RunnableLamda
chain = RunnablePassthrough() | RunnableLambda(russian_lastname)


In [10]:
#
chain.invoke("Paulson")

'Paulsonovich'

### RunnableParallel
We will use RunnableParallel() for running tasks in parallel.
This is probably the most important and most useful Runnable from LangChain.
In the following chain, RunnableParallel is going to run these two tasks in parallel:
* operation_a will use RunnablePassthrough.
* operation_b will use RunnableLambda with the russian_lastname function.

In [23]:
# Running these two tasks in parallel --> good for chatbots
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [16]:
chain.invoke("Douglas")

{'operation_a': 'Douglas', 'operation_b': 'Douglasovich'}

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [25]:
prompt = ChatPromptTemplate.from_template("Tell me a  curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [26]:
# custom function that uses a function
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [36]:
# Building Multiple Runnables
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | llamaChatModel | output_parser

In [38]:
# execution of the parallel runnable function
chain.invoke({
    "name1": "Jordan",  #This name key will not be picked by the parallel runnable function it expects dic key -"name"
    "name": "Abram"
}) 

"A curious fact about Roman Abramovich!\n\nHere's one: Did you know that Roman Abramovich, the Russian billionaire and owner of Chelsea FC, has a fascination with... wait for it... **iceberg-hauling**?!\n\nIn 2012, Abramovich invested in a company called Rosnefteflot, which aimed to tow icebergs from the Antarctic to regions in need of fresh water, such as the Middle East and Africa. The idea was to harvest the icebergs' freshwater content to alleviate water scarcity issues. While the project was met with skepticism and environmental concerns, it showcases Abramovich's unconventional thinking and entrepreneurial spirit.\n\nWho knew that beneath the surface of a football mogul lies a passion for iceberg-hauling?"

### Advanced use of RunnableParrell

In [39]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["paulNtalo focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("What is paulNtalo?")

'paulNtalo is a provider of content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English.'

### *itemgetter* func for RunnableParallel
Use  it to get the multiple input_variables from a template. it has 3 different methods. {...} being the most used.
*  {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }

In [40]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-3.5-turbo")

vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "What is dswithbappy?", "language": "Pirate English"})

"Arrr, dswithbappy be a scallywag focusin' on providin' content on Data Science, AI, ML, DL, CV, NLP, Python programmin', etc. in English. Aye!"